In [1]:
#Step 1 Pre-process dataset 
#Remove spaces from files
import os
PATH=r"D:\Jay\VACC_all\Ferige_Praat"
files = os.listdir(PATH)
for root, dirs, files in os.walk(PATH):
    for i in dirs:
        required=i.replace(" ", "")
        if(required != i):
            os.replace(os.path.join(PATH, i),os.path.join(PATH, required)) 

In [ ]:
#Step 2 Program to merge utterance and time file
#1 folder 20171129Ccx doesnt work for this program so remove
import os
import sys
import glob
import os.path
import pandas as pd
list_of_files = sorted(glob.glob(r'D:\Jay\VACC_all\Ferige_Praat\**\*.txt',recursive=True)) 

for i in range(0,len(list_of_files),2):
    filename = list_of_files[i]
    print(filename)
    filename1 = list_of_files[i+1]
    print(filename1)
    out_name=filename1+"turns_revised.txt"
    csv_input = pd.read_csv(filename,delimiter='\t', usecols=[0,1,2], names=['Start','Stop','Speaker'],header=None)
    csv_input[['Start','Stop']] = csv_input[['Start','Stop']].astype(str)
    csv_input1 = pd.read_csv(filename1,delimiter='\t', usecols=[0,1,2], names=['Start1','Stop1','Utterance'],header=None)
    csv_input1[['Start1','Stop1']] = csv_input1[['Start1','Stop1']].astype(str)
    f = pd.merge(left=csv_input, right=csv_input1, left_on=['Start','Stop'], right_on=['Start1','Stop1'])
    f.drop(['Start1', 'Stop1'], axis=1, inplace=True)
    f = f[['Start','Stop','Speaker','Utterance']]
    f[['Start','Stop']] = f[['Start','Stop']].astype(str)
    f['obj1_count'] = (f['Speaker'].ne(f['Speaker'].shift())).cumsum()
    df=f.groupby('obj1_count').agg(lambda x: ' '.join(x))
    df['Start'] = df['Start'].map(lambda x: x.split(" ")[0])
    df['Stop'] = df['Stop'].map(lambda x: x.split(" ")[-1])
    df['Speaker'] = df['Speaker'].map(lambda x: x.split(" ")[-1])
    if df.iloc[-1]['Speaker'] == df.iloc[0]['Speaker']:
        df1=df.drop(df.index[len(df)-1])
        df1.to_csv(out_name, index=False,header=False,sep='\t')
        del df1
    else:
        df.to_csv(out_name, index=False,header=False,sep='\t')
    del csv_input,csv_input1,f,df


In [3]:
#Step 3 Renaming files with parent folder name
import os
Parent = r'D:\Jay\VACC_all\Ferige_Praat'
for root, dirs, files in os.walk(Parent):
    if not files:
        continue
    prefix = os.path.basename(root)
    for f in files:
        os.rename(os.path.join(root, f), os.path.join(root, "{}_{}".format(prefix, f)))

In [4]:
#Step 4 Remove files which are not revised
import glob
import os
for f in glob.glob(r"D:\Jay\VACC_all\Ferige_Praat\**\*.txt"): # find all csv files
    if not f.endswith("revised.txt"):
        os.remove(f)  # if file name ends in 0.csv, delete it

In [12]:
#Step 5 Rename revised files with underscore
import os
Parent = r'C:\Users\kejri\OneDrive\Desktop\Jay\VACC_all\Ferige_Praat'
for root, dirs, files in os.walk(Parent):
    if not files:
        continue
    prefix = os.path.basename(root)
    for f in files:
        os.rename(os.path.join(root, f), os.path.join(root, "{}_{}".format(prefix, f)))

In [ ]:
# Step 6 Extract semantic embeddings
import pandas as pd
import glob
import os
import sys
import torch
import numpy as np
from functools import reduce
import csv
from sentence_transformers import SentenceTransformer
sen_w_feats = []
sentence_embeddings = []


# Load the BERT tokenizer.
print('Loading Transformer...')
model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')

list_of_files = glob.glob(r'D:\Jay\VACC_all\Ferige_Praat\**\*.txt',recursive=True) 
output_path = r'D:\Jay\DNN\VACC\Embeddings\Text_semantic'

for file_name in list_of_files:
    out_name= os.path.join(output_path, os.path.basename(file_name))
    csv_input = pd.read_csv(file_name, usecols=[3], names=['utterance'],delimiter='\t',header=None)
    for index, row in csv_input.iterrows():
        sen_w_feats.append(row["utterance"])
        
    #Convert sentence to list
    sentence_embeddings = model.encode(sen_w_feats)
    sentence_vectors1=sentence_embeddings.tolist()

    #Merge consecutive utterance of Speaker A and B
    out = reduce(lambda x, y: x+y, sentence_vectors1)

    #Each consecutive utterance is of size 1536 i.e 768 for each utterance
    chunks = [out[x:x+1536] for x in range(0, len(out)-768, 768)]

    #Convert list to array
    arr = np.asarray(chunks)
    with open(out_name, 'w') as fcsv:
        writer = csv.writer(fcsv)
        writer.writerows(arr)
    sen_w_feats = []
    sentence_embeddings = []
    sentence_vectors1=None
    arr=None
    model_output=None

In [2]:
#Step 7 Create h5 files from embeddings extracted from previous step
# The h5 files are needed for training DNN models
import csv
import h5py
import numpy as np
import pandas as pd
import glob
import random
import pdb

SEED=448
frac_train = 1.0



# Create h5 files

#Input path
sessList= sorted(glob.glob(r'D:\Jay\DNN\VACC\Embeddings\Text_semantic\*.txt',recursive=True))

num_files_all = len(sessList)
num_files_train = int(np.ceil((frac_train*num_files_all)))

sessTrain = sessList[:num_files_train]

# Create Train Data file

X_train =np.array([])
X_train = np.empty(shape=(0, 0), dtype='float64' )
for sess_file in sessTrain:
    df_i = pd.read_csv(sess_file)
    xx=np.array(df_i)
    X_train=np.vstack([X_train, xx]) if X_train.size else xx


X_train = X_train.astype('float64')
#Output path
hf = h5py.File(r'D:\Jay\DNN\VACC\h5\semantic\train_nonorm.h5', 'w')
hf.create_dataset('textdataset', data=X_train)
hf.close()




In [ ]:
#Step 8 Create two groups
#Program to split each h5 data into two groups
#Even group represents turns spoken by speaker followed by Alexa
#Odd group represents turns spoken by Alexa followed by a speaker

import h5py
import numpy as np
import pandas as pd

#Input path
path= r'D:\Jay\DNN\VACC\h5\semantic\train_nonorm.h5'
#Output path
new_path = r'D:\Jay\DNN\VACC\h5\semantic\train_nonorm_even.h5'
new_path1 = r'D:\Jay\DNN\VACC\h5\semantic\train_nonorm_odd.h5'

with h5py.File(path, 'r') as f:
   data_set = f['textdataset']
   new_data_even = data_set[::2]
   new_data_odd = data_set[1::2]

with h5py.File(new_path, 'w') as f:
   f.create_dataset('textdataset', data=new_data_even)

with h5py.File(new_path1, 'w') as f:
   f.create_dataset('textdataset', data=new_data_odd)